In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI Model Garden - LLaMA2 (Evaluation)

<table><tbody><tr>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fvertex-ai-samples%2Fmain%2Fnotebooks%2Fcommunity%2Fmodel_garden%2Fmodel_garden_pytorch_llama2_evaluation.ipynb">
      <img alt="Google Cloud Colab Enterprise logo" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" width="32px"><br> Run in Colab Enterprise
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_garden/model_garden_pytorch_llama2_evaluation.ipynb">
      <img alt="GitHub logo" src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" width="32px"><br> View on GitHub
    </a>
  </td>
</tr></tbody></table>

## Overview

This notebook demonstrates downloading prebuilt [LLaMA2 models](https://huggingface.co/meta-llama) and evaluating LLaMA2 models with popular benchmark datasets through Vertex CustomJobs using [EleutherAI's evaluation harness](https://github.com/EleutherAI/lm-evaluation-harness).

### Objective

- Download prebuilt LLaMA2 models
- Evaluate the LLaMA2 models on any of the benchmark datasets

### Costs

This tutorial uses billable components of Google Cloud:

* Vertex AI
* Cloud Storage

Learn about [Vertex AI pricing](https://cloud.google.com/vertex-ai/pricing), [Cloud Storage pricing](https://cloud.google.com/storage/pricing) and use the [Pricing Calculator](https://cloud.google.com/products/calculator/) to generate a cost estimate based on your projected usage.

## Run the notebook

In [ ]:
# @title Setup Google Cloud project

# @markdown 1. [Make sure that billing is enabled for your project](https://cloud.google.com/billing/docs/how-to/modify-project).

# @markdown 2. [Optional] [Create a Cloud Storage bucket](https://cloud.google.com/storage/docs/creating-buckets) for storing
# @markdown experiment outputs. Set the BUCKET_URI for the experiment environment. The specified Cloud Storage bucket (`BUCKET_URI`)
# @markdown should be located in the same region as where the notebook was launched. Note that a multi-region bucket (eg. "us") is
# @markdown not considered a match for a single region covered by the multi-region range (eg. "us-central1").
# @markdown If not set, a unique GCS bucket will be created instead.

# Import the necessary packages
import json
import os
import sys
from datetime import datetime

from google.cloud import aiplatform, storage

# Get the default cloud project id.
PROJECT_ID = os.environ["GOOGLE_CLOUD_PROJECT"]

# Get the default region for launching jobs.
REGION = os.environ["GOOGLE_CLOUD_REGION"]

# Enable the vertex AI API and Compute Engine API, if not already.
print("Enabling vertex AI API and Compute Engine API.")
! gcloud services enable aiplatform.googleapis.com compute.googleapis.com

# Cloud Storage bucket for storing the experiment artifacts.
# A unique Google Cloud bucket will be created for the purpose of this notebook. If you
# prefer using your own Google Cloud bucket, please change the value yourself below.
now = datetime.now().strftime("%Y%m%d%H%M%S")
BUCKET_URI = "gs://"  # @param {type:"string"}
assert BUCKET_URI.startswith("gs://"), "BUCKET_URI must start with `gs://`."
if BUCKET_URI is None or BUCKET_URI.strip() == "" or BUCKET_URI == "gs://":
    # Create a unique Google Cloud bucket for this notebook if not specified
    BUCKET_URI = f"gs://{PROJECT_ID}-tmp-{now}"
    ! gsutil mb -l {REGION} {BUCKET_URI}
else:
    BUCKET_NAME = "/".join(BUCKET_URI.split("/")[:3])

    shell_output = ! gsutil ls -Lb {BUCKET_NAME} | grep "Location constraint:" | sed "/sLocation constraint: //"
    bucket_region = shell_output[0].strip().lower()
    if bucket_region != REGION:
        raise ValueError(
            "Bucket region %s is different from the notebook region %s."
            % (bucket_region, REGION)
        )
print(f"Using this GCS Bucket: {BUCKET_URI}")

# Gets the default SERVICE_ACCOUNT.
shell_output = ! gcloud projects describe $PROJECT_ID
project_number = shell_output[-1].split(":")[1].strip().replace("'", "")
SERVICE_ACCOUNT = f"{project_number}-compute@developer.gserviceaccount.com"

print("Using this default Service Account:", SERVICE_ACCOUNT)
print(f"Using this Google Cloud Bucket: {BUCKET_URI}")

# Provision permissions to the SERVICE_ACCOUNT with the Google Cloud bucket
BUCKET_NAME = "/".join(BUCKET_URI.split("/")[:3])
! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.admin $BUCKET_NAME


! gcloud config set project $PROJECT_ID


STAGING_BUCKET = os.path.join(BUCKET_URI, "temporal")
EXPERIMENT_BUCKET = os.path.join(BUCKET_URI, "peft")
BASE_MODEL_BUCKET = os.path.join(EXPERIMENT_BUCKET, "base_model")
MODEL_BUCKET = os.path.join(EXPERIMENT_BUCKET, "model")

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user(project_id=PROJECT_ID)

aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=STAGING_BUCKET)

# The evaluation docker image.
EVAL_DOCKER_URI = "us-docker.pkg.dev/vertex-ai/vertex-vision-model-garden-dockers/pytorch-lm-evaluation-harness:20231011_0934_RC00"


def get_job_name_with_datetime(prefix: str) -> str:
    """Gets the job name with date time when triggering training or deployment
    jobs in Vertex AI.
    """
    return prefix + datetime.now().strftime("_%Y%m%d_%H%M%S")

In [ ]:
# @title Access pretrained Code LLaMA models

# @markdown The original models from Meta are converted into the HuggingFace format for serving in Vertex AI.

# @markdown Accept the model agreement to access the models:
# @markdown 1. Open the [Code LLaMA model card](https://console.cloud.google.com/vertex-ai/publishers/google/model-garden/137).
# @markdown 2. Review and accept the agreement in the pop-up window on the model card page. If you have previously accepted the model agreement, there will not be a pop-up window on the model card page and this step is not needed.
# @markdown 3. A Cloud Storage bucket (starting with ‘gs://’) containing LLaMA 2 pretrained and finetuned models will be shared under the “Documentation” section and its “Get started” subsection.

# This path will be shared once click the agreement in Code LLaMA model card
# as described in the `Access pretrained Code LLaMA models` section.
VERTEX_AI_MODEL_GARDEN_LLAMA2 = "gs://"  # @param {type: "string"}  # This will be shared once click the agreement of LLaMA2 in Vertex AI Model Garden.
assert (
    VERTEX_AI_MODEL_GARDEN_LLAMA2
), "Please click the agreement of LLaMA2 in Vertex AI Model Garden, and get the GCS path of LLaMA2 model artifacts."
print(
    "Copy LLaMA2 model artifacts from",
    VERTEX_AI_MODEL_GARDEN_LLAMA2,
    "to ",
    BASE_MODEL_BUCKET,
)
! gsutil -m cp -R $VERTEX_AI_MODEL_GARDEN_LLAMA2/* $BASE_MODEL_BUCKET

In [ ]:
# @title Evaluate PEFT-finetuned LLaMA 2 models
# @markdown This section demonstrates evaluation of LLaMA2 models using EleutherAI's [Language Model Evaluation Harness (lm-evaluation-harness)](https://github.com/EleutherAI/lm-evaluation-harness) with Vertex Custom Job.

# @markdown This example uses the dataset [TruthfulQA](https://arxiv.org/abs/2109.07958).
# @markdown All the supported tasks are listed in [this task table](https://github.com/EleutherAI/lm-evaluation-harness/blob/master/docs/task_table.md).

# set the base model id
base_model_name = "llama2-7b-chat-hf"  # @param ["llama2-7b-hf", "llama2-7b-chat-hf", "llama2-13b-hf", "llama2-13b-chat-hf", "llama2-70b-hf", "llama2-70b-chat-hf"]
base_model_id = os.path.join(BASE_MODEL_BUCKET, base_model_name)

# Set the machine_type, accelerator_type, accelerator_count and benchmark dataset.
eval_dataset = "truthfulqa_mc"  # @param ["truthfulqa_mc", "boolq", "gsm8k", "hellaswag", "natural_questions", "openai_humaneval", "openbookqa", "quac", "trivia_qa", "winograde"]

# Worker pool spec.
# Find Vertex AI supported accelerators and regions in:
#  https://cloud.google.com/vertex-ai/docs/training/configure-compute

if base_model_name == "llama2-7b-hf":
    # Sets 1 (24G) to evaluate LLaMA2 7B models.
    machine_type = "g2-standard-16"
    accelerator_type = "NVIDIA_L4"
    accelerator_count = 1
elif base_model_name == "llama2-7b-chat-hf":
    # Sets 1 L4 (24G) to evaluate LLaMA2 7B models.
    machine_type = "g2-standard-16"
    accelerator_type = "NVIDIA_L4"
    accelerator_count = 1
elif base_model_name == "llama2-13b-hf":
    # Sets 2 L4 (24G) to evaluate LLaMA2 13B models.
    machine_type = "g2-standard-24"
    accelerator_type = "NVIDIA_L4"
    accelerator_count = 2
elif base_model_name == "llama2-13b-chat-hf":
    # Sets 2 L4 (24G) to evaluate LLaMA2 13B models.
    machine_type = "g2-standard-24"
    accelerator_type = "NVIDIA_L4"
    accelerator_count = 2
elif base_model_name == "llama2-70b-hf":
    # Sets 8 L4 (24G) to evaluate LLaMA2 70B models.
    machine_type = "g2-standard-96"
    accelerator_type = "NVIDIA_L4"
    accelerator_count = 8
elif base_model_name == "llama2-70b-chat-hf":
    # Sets 8 L4 (24G) to evaluate LLaMA2 70B models.
    machine_type = "g2-standard-96"
    accelerator_type = "NVIDIA_L4"
    accelerator_count = 8

replica_count = 1

job_name = get_job_name_with_datetime(prefix="llama2-eval")
eval_output_dir = os.path.join(MODEL_BUCKET, job_name)
eval_output_dir_gcsfuse = eval_output_dir.replace("gs://", "/gcs/")
base_model_id_gcsfuse = base_model_id.replace("gs://", "/gcs/")

# @markdown  To evaluate a PEFT-finetuned model, enter the PEFT output directory below.
# @markdown  Otherwise, leave it empty.

# @markdown  See the finetuning notebook for more details:

# @markdown  https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_garden/model_garden_pytorch_llama2_peft_finetuning.ipynb
peft_output_dir = ""  # @param {type:"string"}
peft_output_dir_gcsfuse = peft_output_dir.replace("gs://", "/gcs/")

# Prepare evaluation command that runs the evaluation harness.
# Set `trust_remote_code = True` because evaluating the model requires
# executing code from the model repository.
# Set `use_accelerate = True` to enable evaluation across multiple GPUs.
eval_command = [
    "python",
    "main.py",
    "--model",
    "hf-causal-experimental",
    "--tasks",
    f"{eval_dataset}",
    "--output_path",
    f"{eval_output_dir_gcsfuse}",
]

if peft_output_dir_gcsfuse:
    eval_command += [
        "--model_args",
        f"pretrained={base_model_id_gcsfuse},peft={peft_output_dir_gcsfuse},trust_remote_code=True,use_accelerate=True,device_map_option=auto",
    ]
else:
    eval_command += [
        "--model_args",
        f"pretrained={base_model_id_gcsfuse},trust_remote_code=True,use_accelerate=True,device_map_option=auto",
    ]

# Run the evaluation job.
worker_pool_specs = [
    {
        "machine_spec": {
            "machine_type": machine_type,
            "accelerator_type": accelerator_type,
            "accelerator_count": accelerator_count,
        },
        "replica_count": replica_count,
        "disk_spec": {
            "boot_disk_size_gb": 500,
        },
        "container_spec": {
            "image_uri": EVAL_DOCKER_URI,
            "command": eval_command,
            "args": [],
        },
    }
]

eval_job = aiplatform.CustomJob(
    display_name=job_name,
    worker_pool_specs=worker_pool_specs,
    base_output_dir=eval_output_dir,
)

eval_job.run()

print("Evaluation results were saved in:", eval_output_dir)

# Fetch evaluation results.
storage_client = storage.Client()
BUCKET_NAME = BUCKET_URI.replace("gs://", "")
bucket = storage_client.get_bucket(BUCKET_NAME)
RESULT_FILE_PATH = eval_output_dir[len(BUCKET_URI) + 1 :]
blob = bucket.blob(RESULT_FILE_PATH)
raw_result = blob.download_as_string()

# Print evaluation results.
result = json.loads(raw_result)
result_formatted = json.dumps(result, indent=2)
print(f"Evaluation result:\n{result_formatted}")

In [ ]:
# @title Clean up resources
# @markdown  Delete the experiment models and endpoints to recycle the resources
# @markdown  and avoid unnecessary continouous charges that may incur.

eval_job.delete()

# Delete Cloud Storage objects that were created
delete_bucket = False  # @param {type: "boolean"}
if delete_bucket:
    ! gsutil -m rm -r $EXPERIMENT_BUCKET